In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df_train=pd.read_csv("/content/drive/MyDrive/Minor Project 2/Instacart_history (1).csv")
# df_train=df_train.head(10000)

In [ ]:
# Mapping values
def encode(df,i):
  item_unique = df[i].unique().tolist()
  item_map = dict(zip(item_unique, range(1,len(item_unique) + 1)))
  item_map[-1] = 0
  df[i] = df[i].apply(lambda x: item_map[x])

In [ ]:
# Renaming Columns
df_train.rename(columns = {'ORDER_NUMBER':'TRANSACTION_DT','MATERIAL_NUMBER':'PRODUCT_ID'}, inplace = True)

In [ ]:
df_train['AMOUNT']=1
df_train=df_train.sort_values(by=['CUSTOMER_ID','TRANSACTION_DT'])
df_train

,CUSTOMER_ID,TRANSACTION_DT,PRODUCT_ID,AMOUNT
0,1,1,1,1
1,1,1,2,1
2,1,1,3,1
3,1,1,4,1
4,1,1,5,1
...,...,...,...,...
1411001,19935,50,507,1
1411002,19935,50,30,1
1411003,19935,50,4984,1
1411004,19935,50,7,1


In [ ]:
# df_train=df_train.sort_values(by=['CUSTOMER_ID','TRANSACTION_DT'])

In [ ]:
# df_train

In [ ]:
# Storing customer and product count
customer=df_train['CUSTOMER_ID'].unique()
product=df_train['PRODUCT_ID'].unique()

In [ ]:
# Selecting Users with 40 Baskets
for i in customer:
  if df_train[df_train['CUSTOMER_ID']==i]['TRANSACTION_DT'].unique().size<40:
    df_train=df_train.drop(df_train[df_train['CUSTOMER_ID'] == i].index)


In [ ]:
df_train

,CUSTOMER_ID,TRANSACTION_DT,PRODUCT_ID,AMOUNT
1031,27,1,544,1
1032,27,2,545,1
1033,27,2,546,1
1034,27,2,547,1
1035,27,2,548,1
...,...,...,...,...
1411001,19935,50,507,1
1411002,19935,50,30,1
1411003,19935,50,4984,1
1411004,19935,50,7,1


In [ ]:
#  df_train['PRODUCT_ID'].unique().size

In [ ]:
grouped_df=df_train.groupby(['CUSTOMER_ID', 'TRANSACTION_DT', 'PRODUCT_ID'])['AMOUNT'].sum().reset_index()
pivot_df = grouped_df.pivot(index=['CUSTOMER_ID', 'TRANSACTION_DT'], columns='PRODUCT_ID', values='AMOUNT').fillna(0)
pivot_df

In [ ]:
# pivot_df = grouped_df.pivot(index=['CUSTOMER_ID', 'TRANSACTION_DT'], columns='PRODUCT_ID', values='AMOUNT').fillna(0)

In [ ]:
# pivot_df

In [ ]:
pivot_df.describe()

In [ ]:
pivot_df.columns

In [ ]:
pivot_df.reset_index(inplace=True)
pivot_df.columns.name = None
pivot_df

In [ ]:
# pivot_df

In [ ]:
# Again Storing Customer and Product count
customer=df_train['CUSTOMER_ID'].unique()
product=df_train['PRODUCT_ID'].unique()

In [ ]:
# Dividing Dataset for Training Testing and Validation
train_basket=[]     # Training basket
validate_basket=[]  # Validation basket
test_basket=[]      # Testing basket
actual_basket=[]    # Actual basket
customer=pivot_df['CUSTOMER_ID'].unique()
for i in pivot_df['CUSTOMER_ID'].unique():
  train=[]
  validate=[]
  test=[]
  actual=[]
  df=pivot_df[pivot_df['CUSTOMER_ID']==i]
  count=0
  size=df['TRANSACTION_DT'].unique().size
  date=df['TRANSACTION_DT'].unique()
  for j in df['TRANSACTION_DT'].unique():
    if count<size-2:
      df3=df[df['TRANSACTION_DT']==j]
      df3.drop(['CUSTOMER_ID','TRANSACTION_DT'], inplace=True, axis=1)
      arr=df3.to_numpy()
      arr=arr.tolist()
      train.append(arr[0])

    if count>=1 and count<size-1:
      df3=df[df['TRANSACTION_DT']==j]
      df3.drop(['CUSTOMER_ID','TRANSACTION_DT'], inplace=True, axis=1)
      arr=df3.to_numpy()
      arr=arr.tolist()
      validate.append(arr[0])

    if count==size-2:
      df3=df[df['TRANSACTION_DT']==j]
      df3.drop(['CUSTOMER_ID','TRANSACTION_DT'], inplace=True, axis=1)
      arr=df3.to_numpy()
      arr=arr.tolist()
      test.append(arr[0])

    if count==size-1:
      df3=df[df['TRANSACTION_DT']==j]
      df3.drop(['CUSTOMER_ID','TRANSACTION_DT'], inplace=True, axis=1)
      arr=df3.to_numpy()
      arr=arr.tolist()
      actual.append(arr[0])

    count+=1
  validate_basket.append(validate)
  train_basket.append(train)
  test_basket.append(test)
  actual_basket.append(actual)

Streaming output truncated to the last 5000 lines.
<ipython-input-20-489057ca162e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.drop(['CUSTOMER_ID','TRANSACTION_DT'], inplace=True, axis=1)
<ipython-input-20-489057ca162e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.drop(['CUSTOMER_ID','TRANSACTION_DT'], inplace=True, axis=1)
<ipython-input-20-489057ca162e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.drop(['C

In [ ]:
########################### 1st Model UGP using FFNN #####################################################

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Padding Dataset with zeros in front
max_len = max(len(sublist) for sublist in train_basket)
print(max_len)
padded_data = [sublist + [[0.0] * len(sublist[0])] * (max_len - len(sublist)) for sublist in train_basket]
train_tensor = torch.tensor(padded_data, dtype=torch.float32)
max_len = max(len(sublist) for sublist in validate_basket)
print(max_len)
padded_data = [sublist + [[0.0] * len(sublist[0])] * (max_len - len(sublist)) for sublist in validate_basket]
validate_tensor = torch.tensor(padded_data, dtype=torch.float32)
test_tensor = torch.tensor(test_basket, dtype=torch.float32)
actual_tensor = torch.tensor(actual_basket, dtype=torch.float32)

48
48


In [ ]:
train_tensor.shape

torch.Size([298, 48, 6204])

In [ ]:
# validate_tensor.shape

In [ ]:
# test_tensor.shape

In [ ]:
# Model Initialization
model = nn.Sequential(
    nn.Linear(6204, 128),
    nn.ReLU(),
    nn.Linear(128, 256),
    nn.ReLU(),
    nn.Linear(256, 6204),
    nn.Sigmoid())

In [ ]:
# Loss Function and Optimizer Selection
loss_fn =  nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Model Training
n_epochs = 25
batch_size = 1
X=train_tensor
y=validate_tensor
for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.2334076464176178
Finished epoch 1, latest loss 0.2319638878107071
Finished epoch 2, latest loss 0.22780512273311615
Finished epoch 3, latest loss 0.22377704083919525
Finished epoch 4, latest loss 0.21951846778392792
Finished epoch 5, latest loss 0.21518224477767944
Finished epoch 6, latest loss 0.21236300468444824
Finished epoch 7, latest loss 0.20978587865829468
Finished epoch 8, latest loss 0.20894518494606018
Finished epoch 9, latest loss 0.20624294877052307
Finished epoch 10, latest loss 0.20487481355667114
Finished epoch 11, latest loss 0.20463183522224426
Finished epoch 12, latest loss 0.20366650819778442
Finished epoch 13, latest loss 0.2017984837293625
Finished epoch 14, latest loss 0.20070742070674896
Finished epoch 15, latest loss 0.19996050000190735
Finished epoch 16, latest loss 0.19955308735370636
Finished epoch 17, latest loss 0.19818058609962463
Finished epoch 18, latest loss 0.19784976541996002
Finished epoch 19, latest loss 0.19754967093

In [ ]:
# Model Prediction
y_pred = model(test_tensor)
y_pred

tensor([[[1.0628e-04, 2.6471e-05, 9.6392e-01,  ..., 1.0000e+00,
          9.9931e-01, 9.9910e-01]],

        [[1.4238e-02, 3.8398e-01, 6.3029e-10,  ..., 1.3840e-01,
          1.0341e-02, 9.9980e-01]],

        [[9.9999e-01, 7.8628e-03, 2.4771e-04,  ..., 6.7964e-01,
          9.9784e-01, 9.9439e-01]],

        ...,

        [[1.4716e-01, 4.8281e-04, 5.2642e-03,  ..., 9.1200e-01,
          2.2489e-01, 5.6200e-01]],

        [[5.5688e-01, 9.1743e-04, 6.8244e-01,  ..., 5.7536e-02,
          2.4075e-02, 2.5716e-02]],

        [[9.8184e-02, 1.6252e-01, 2.8245e-01,  ..., 5.3527e-01,
          1.6104e-02, 3.8801e-01]]], grad_fn=<SigmoidBackward0>)

In [ ]:
# y_pred

In [ ]:
# y_pred.shape

In [ ]:
# Converion from 3d to 2d
y_pred_2d = y_pred.view(298, 6204)
y_pred_2d

tensor([[1.0628e-04, 2.6471e-05, 9.6392e-01,  ..., 1.0000e+00, 9.9931e-01,
         9.9910e-01],
        [1.4238e-02, 3.8398e-01, 6.3029e-10,  ..., 1.3840e-01, 1.0341e-02,
         9.9980e-01],
        [9.9999e-01, 7.8628e-03, 2.4771e-04,  ..., 6.7964e-01, 9.9784e-01,
         9.9439e-01],
        ...,
        [1.4716e-01, 4.8281e-04, 5.2642e-03,  ..., 9.1200e-01, 2.2489e-01,
         5.6200e-01],
        [5.5688e-01, 9.1743e-04, 6.8244e-01,  ..., 5.7536e-02, 2.4075e-02,
         2.5716e-02],
        [9.8184e-02, 1.6252e-01, 2.8245e-01,  ..., 5.3527e-01, 1.6104e-02,
         3.8801e-01]], grad_fn=<ViewBackward0>)

In [ ]:
# y_pred_2d

In [ ]:

# rec=0
# for i in range(len(actual_basket)):
#   r=0
#   x=0
#   for j in range(len(actual_basket[0])):
#     for k in range(len(actual_basket[0][0])):
#       if((actual_basket[i][j][k]==1 and y_pred[i][j][k]>=0.6)):
#         r+=1
#       if(actual_basket[i][j][k]==1):
#         x+=1
#   rec=rec+r/x

# recall=rec/len(actual_basket)
# print(recall)

In [ ]:
# len(actual_basket[0][0])

In [ ]:
#recall @ 5
# top_recommendation=[]
# for i in range(len(actual_basket)):
#   top_recommendation.append(y_pred[i][0].topk(5,dim=0).indices)

# rec=0
# for i in range(len(actual_basket)):
#   r=0
#   for k in range(5):
#     if(actual_basket[i][0][top_recommendation[i][k]]==1):
#       r+=1
#   rec=rec+r/5

# recall=rec/len(actual_basket)
# print(recall)

In [ ]:
def find_recall(k):
  top_recommendation=[]
  for i in range(len(actual_basket)):
    top_recommendation.append(y_pred[i][0].topk(k,dim=0).indices)

  rec=0
  for i in range(len(actual_basket)):
    r=0
    x=0
    for j in range(k):
      if(actual_basket[i][0][top_recommendation[i][j]]==1):
        r+=1
    for j in range(len(actual_basket[i][0])):
      if(actual_basket[i][0][j]==1):
        x+=1
    rec=rec+r/x

  recall=rec/len(actual_basket)
  return recall

In [ ]:
print(find_recall(5))
print(find_recall(10))
print(find_recall(15))

0.0163022787250446
0.022121957727857338
0.029715026478644213


In [ ]:
def find_precision(k):
  top_recommendation=[]
  for i in range(len(actual_basket)):
    top_recommendation.append(y_pred[i][0].topk(k,dim=0).indices)

  prec=0
  for i in range(len(actual_basket)):
    r=0
    for j in range(k):
      if(actual_basket[i][0][top_recommendation[i][j]]==1):
        r+=1
    prec=prec+r/k

  precision=prec/len(actual_basket)
  return precision

In [ ]:
print(find_precision(5))
print(find_precision(10))
print(find_precision(15))

0.02416107382550337
0.018456375838926165
0.015436241610738257


In [ ]:
#F1 Score
print((2*find_precision(5)*find_recall(5))/(find_recall(5)+find_precision(5)))
print((2*find_precision(10)*find_recall(10))/(find_recall(10)+find_precision(10)))
print((2*find_precision(15)*find_recall(15))/(find_recall(15)+find_precision(15)))

0.019468508414258987
0.020123604408062237
0.0203178492035177


In [ ]:
########################################################### 2nd Model LSTM     ####################################################################

In [ ]:
#normal lstm
import torch
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available, and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming you have your train data loaded as a tensor
# Ensure that train_data has the shape (num_users, sequence_length, input_features)
train_data = train_tensor

# Define the batch size and other constants
batch_size = 8  # Adjust as needed
sequence_length = train_tensor.shape[1]
input_features = train_tensor.shape[2]
output_features = train_tensor.shape[2]  # Number of features in the predicted sequence (next sequence)

# Create DataLoader for training data
train_dataset = torch.utils.data.TensorDataset(train_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the LSTM model with the correct input and output size
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.linear(lstm_out)
        output = torch.sigmoid(output)
        #print(output)
        return output

# Initialize the model and move it to the GPU
model_LSTM = LSTMModel(input_features, 64, 2, output_features).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20  # Adjust as needed
for epoch in range(num_epochs):
    for batch in train_loader:
        # Move the batch to the GPU
        batch = batch[0].to(device)  # Unwrap the batch from the DataLoader

        # Split the input and target sequences
        input_seq = batch[:, :-1, :]  # Input sequence (exclude the last time step)
        target_seq = batch[:, 1:, :]   # Target sequence (exclude the first time step)

        # Forward pass
        train_predictions = model(input_seq)

        # Compute loss for this batch
        loss = criterion(train_predictions, target_seq)  # Predicting the next sequence

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss for monitoring at the end of each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Generate predictions for each user
user_predictions = []
with torch.no_grad():
    for user_data in train_loader:
        user_data = user_data[0].to(device)
        user_input_seq = user_data[:, :, :]  # Input sequence (exclude the last time step)
        user_prediction = model(user_input_seq)
        user_predictions.append(user_prediction.cpu().numpy())

# Delete the model to save memory


Epoch [1/20], Loss: 0.0025803036987781525
Epoch [2/20], Loss: 0.0009246536646969616
Epoch [3/20], Loss: 0.0007235996890813112
Epoch [4/20], Loss: 0.001712903962470591
Epoch [5/20], Loss: 0.0013188659213483334
Epoch [6/20], Loss: 0.0018345109419897199
Epoch [7/20], Loss: 0.0011311097769066691
Epoch [8/20], Loss: 0.0011673144763335586
Epoch [9/20], Loss: 0.0012728747678920627
Epoch [10/20], Loss: 0.0014172587543725967
Epoch [11/20], Loss: 0.0007603408885188401
Epoch [12/20], Loss: 0.0010205996222794056
Epoch [13/20], Loss: 0.0008583283633925021
Epoch [14/20], Loss: 0.0014897919027134776
Epoch [15/20], Loss: 0.0011443813564255834
Epoch [16/20], Loss: 0.000850495183840394
Epoch [17/20], Loss: 0.0014629224315285683
Epoch [18/20], Loss: 0.0020804572850465775
Epoch [19/20], Loss: 0.0019160144729539752
Epoch [20/20], Loss: 0.0008907722076401114


In [ ]:
# Model Prediction
lstm_res=model_LSTM(test_tensor)

In [ ]:
# lstm_res.shape

In [ ]:
# Conversion from 3d to 2d
lstm_res_2d = lstm_res.view(298, 6204)

In [ ]:
############################################################# 3rd Model KNN ########################################################################

In [ ]:
# Finding K Nearest Neighbours
import numpy as np
from sklearn.neighbors import NearestNeighbors

num_nearest_neighbors = 30

nn_model = NearestNeighbors(n_neighbors = num_nearest_neighbors, metric = 'cosine')

u=y_pred_2d

u = u.detach().numpy()
nn_model.fit(u)

nearest_neighbors_indices = nn_model.kneighbors(u, return_distance=False)

In [ ]:
# nearest_neighbors_indices

In [ ]:
# float(y_pred[0][0][0])

In [ ]:
# y_pred_2d.shape

In [ ]:
# Mean Polling
knn=[]

for i in range(0,len(customer)):
  temp = [0]*len(product)
  for j in range (0, len(product)):
    for k in range(0,30):
      # print(nearest_neighbors_indices[i][k])
      x=float(y_pred_2d[nearest_neighbors_indices[i][k]][j])
      temp[j]=temp[j]+x
    temp[j]=temp[j]/20
  knn.append(temp)



In [ ]:
# type(y_pred_2d)

In [ ]:
# Converting to Numpy
y_pred_2d_numpy=y_pred_2d.detach().cpu().numpy()
lstm_res_2d_numpy=lstm_res_2d.detach().cpu().numpy()
knn_numpy=np.array(knn)
actual_basket_numpy=np.array(actual_basket)

# Conversion from 3d to 2d
actual_basket_numpy_2d = actual_basket_numpy.reshape(298,6204)

In [ ]:
# type(y_pred_2d_numpy)

In [ ]:
# type(lstm_res_2d)

In [ ]:
# lstm_res_2d_numpy=lstm_res_2d.detach().cpu().numpy()

In [ ]:
# type(lstm_res_2d_numpy)

In [ ]:
# type(knn)

In [ ]:
# knn_numpy=np.array(knn)

In [ ]:
# type(knn_numpy)

In [ ]:
# knn_numpy.shape

In [ ]:
# actual_basket_numpy=np.array(actual_basket)

In [ ]:
# len(actual_basket_numpy_2d)

In [ ]:
# actual_basket_numpy_2d = actual_basket_numpy.reshape(298,6204)

In [ ]:
def find_recall_final(k):
  top_recommendation=[]
  final_prediction_tensor = torch.from_numpy(final_prediction)
  for i in range(len(actual_basket_numpy_2d)):
    top_recommendation.append(final_prediction_tensor[i].topk(k,dim=0).indices)

  rec=0
  for i in range(len(actual_basket_numpy_2d)):
    r=0
    x=0
    for j in range(k):
      if(actual_basket_numpy_2d[i][top_recommendation[i][j]]==1):
        r+=1
    for j in range(len(actual_basket_numpy_2d[i])):
      if(actual_basket_numpy_2d[i][j]==1):
        x+=1
    rec=rec+r/x

  recall=rec/len(actual_basket_numpy_2d)
  return recall

In [ ]:
# HyperParameter Tuning for alpha
for i in range(1,10):
    alpha = i/10
    final_prediction = alpha*(y_pred_2d_numpy + lstm_res_2d_numpy) + (1-alpha) * (knn_numpy)
    print(alpha)
    print(find_recall_final(5))
    print(find_recall_final(10))
    print(find_recall_final(20))
    print(find_recall_final(40))



0.1
0.004718687352915541
0.007732079628052783
0.012857028465470712
0.031017991194876377
0.2
0.004718687352915541
0.007732079628052783
0.012910293619407006
0.03280176856590207
0.3
0.005750699710431254
0.007732079628052783
0.01270522277675227
0.03305477804709948
0.4
0.005750699710431254
0.007732079628052783
0.01270522277675227
0.032014896973661354
0.5
0.006349932692214571
0.007732079628052783
0.012049220354589482
0.0317352549154958
0.6
0.005271313325004599
0.007732079628052783
0.011881435119690155
0.03237443676273134
0.7
0.003953000765081302
0.008690852398906091
0.013852245815333064
0.03137837838412263
0.8
0.001855685328839691
0.007906401950026112
0.011071667025150014
0.035526715571346526
0.9
0.0011845443892423758
0.007527389221101188
0.014344300089213615
0.033741078886380976


In [ ]:
for i in range(1,20):
    alpha = i/10
    final_prediction = alpha*(y_pred_2d_numpy + lstm_res_2d_numpy) + (1-alpha) * (knn_numpy)
    print(alpha)
    print(find_recall_final(5))
    print(find_recall_final(10))
    print(find_recall_final(20))
    print(find_recall_final(40))